# Converting database to pandas DataFrame
```Python
with read_engine.connect() as conn:
    conn.execute(text("SET TRANSACTION ISOLATION LEVEL READ COMMITTED"))

    # Read only the filtered points from the database
    points = gpd.read_postgis(
        sql_query,
        conn,
        geom_col='geometry'
    )

# 'points' contains only the data for trajectories with more than 1 point
points_df = pd.DataFrame(points.drop(columns=['geometry', 'descriptors']))
points_df['x'] = points.geometry.x
points_df['y'] = points.geometry.y

points_df.to_feather('points.feather')
```

In [ ]:
from datetime import datetime
from multiprocessing import Pool

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pysida.lib import GetSatPairs, get_deformation_from_pair, get_nn_distances

DAYS_IN_SECONDS = 24 * 60 * 60

In [ ]:
# Read feather file and convert to a DataFrame usable by pysida
feather_file = 'points.feather'
points = pd.read_feather(feather_file)
df = points[['image_id', 'trajectory_id', 'time', 'x', 'y', 'corr']].copy().rename(columns={'image_id':'i', 'trajectory_id': 'g', 'time':'d', 'x':'x', 'y':'y', 'corr':'q'})
print(df.shape)
df.head()

In [ ]:
# plot all points between two days
sub_df = df[(df.d > datetime(2020, 1, 10)) & (df.d < datetime(2020, 1, 11))]
plt.scatter(sub_df.x, sub_df.y, c=sub_df.g, s=1)
plt.colorbar()
print(sub_df.shape, sub_df.g.unique().size)

In [ ]:
# plot histogram of trajectory lengths between two dates
sub_df = df[(df.d > datetime(2020, 1, 10)) & (df.d < datetime(2020, 1, 15))]
traj_lengths = [grp.shape[0]  for _, grp in sub_df.groupby('g')]
plt.hist(traj_lengths)

In [ ]:
# plot longest trajectories
for _, grp in sub_df.groupby('g'):
    if grp.shape[0] > 25:
        grp_sorted = grp.sort_values('d')        
        plt.plot(grp_sorted.x, grp_sorted.y, '.-')

In [ ]:
# Create a list of Pairs.
# A Pair - contains start and stop points from two images (x0, y0, d0, x1, y1, d1).
# It also contains triangulation, area and perimeter of the faces.

In [ ]:
date_begin = datetime(2020, 1, 10)
date_end = datetime(2020, 1, 15)

# Get pairs for a short period of time
gsp = GetSatPairs(df, min_time_diff = 0.5, max_time_diff = 1.5)
pairs = gsp.get_all_pairs(date_begin, date_end)

In [ ]:
sizes = [pair.x0.size for pair in pairs]
print(len(pairs), min(sizes), max(sizes), pairs[0].__dict__.keys())

In [ ]:
# select the largest pair and plot
pair = pairs[np.argmax(sizes)]
plt.quiver(pair.x0, pair.y0, pair.x1 - pair.x0, pair.y1 - pair.y0, angles='xy', scale_units='xy', scale=0.1)
print(pair.d0, pair.d1)

In [ ]:
# compute deformation for selected pairs
with Pool(4) as pool:
    defors = pool.map(get_deformation_from_pair, pairs)

In [ ]:
# select the same deformation as the largest pair and plot
print(len(pairs), len(defors), defors[0].__dict__.keys())
defor = defors[np.argmax(sizes)]
plt.figure()
plt.tripcolor(pair.x0, pair.y0, pair.t, defor.e2 * DAYS_IN_SECONDS, cmap='plasma_r', mask=~pair.g, clim=(0,0.5))

In [ ]:
for pair, defor in zip(pairs, defors):
    plt.tripcolor(pair.x0, pair.y0, pair.t, defor.e2 * DAYS_IN_SECONDS, cmap='plasma_r', mask=~pair.g, clim=(0,0.5))

In [ ]:
# get sizes and time differences of pairs
sizes = [pair.x0.size for pair in pairs]
time_difs = [(pair.d1 - pair.d0).total_seconds()/ DAYS_IN_SECONDS for pair in pairs]

# plot deformation, drift, histograms of differences between points and deformation
for pair, defor in zip(pairs, defors):
    if pair.d0 == pd.Timestamp('2020-01-13 16:45:16') and pair.d1 == pd.Timestamp('2020-01-14 15:48:02'):
        distances = get_nn_distances(pair)        
        fig, axs = plt.subplots(1, 5, figsize=(18, 4))
        qui0 = axs[0].quiver(pair.x0, pair.y0, pair.x1 - pair.x0, pair.y1 - pair.y0, angles='xy', scale_units='xy', scale=0.1, color='k', width=0.002)
        axs[0].set_aspect('equal')

        trp1 = axs[1].tripcolor(pair.x0, pair.y0, pair.t, defor.e2 * 24 * 60 * 60, clim=[0,0.1], mask=~pair.g, cmap='plasma_r')
        plt.colorbar(trp1, ax=axs[1], shrink=0.5, label='Deformation rate (m/day)')
        axs[1].set_aspect('equal')

        _ = axs[2].hist(distances, bins=50)

        _ = axs[3].hist2d(sizes, time_difs, bins=[50, 50], cmap='jet', cmin=1)

        _ = axs[4].hist(pair.a[pair.g], bins=50, range=[0, 0.5e8])

        plt.show()
        break        